In [33]:
import sys
sys.path.append('../../src/')

In [34]:
%load_ext autoreload
%autoreload 2

from train import train_model
from model import initialize_model
from utils import set_requires_grad, save_model
from data import load_data
from plotting import plot_data_loader
from eval import eval_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [36]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [37]:
data_dir = "../../data"
images_dir = "../../data/chest_xray"

In [38]:
# Get best num_workers
# for i in range(97):
#     start = time.time()
#     data_loader = load_data(images_dir,
#                                                                    batch_size = 96, 
#                                                                    input_size = 299, 
#                                                                    norm_arr = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#                                                                    num_workers = i)
#     iter(data_loader['train']).next()[0].shape
#     print(f"{i}: {time.time()-start}")

In [39]:
# Models options: resnet50, resnet34, inceptionv3, vgg16, mobile_net_v3_large, efficient_net_b1, efficient_net_b0.
model_name = "efficient_net_b1"

# Number of classes.
num_classes = 3

# Batch Size.
batch_size = 16

# Epochs to train for.
num_epochs = 100

# Number of workers for data loader.
num_workers = 0

# Imagenet norm array passed as default value.
norm_arr=([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# Chest x-ray8 training dataset metrics 
norm_arr=([0.4810, 0.4810, 0.4810], [0.2373, 0.2373, 0.2373])


# Feature extract flag: False - Tune the whole model,
#                       True - Update only the reshaped layer parameters.
feature_extract = False

# Use pretrained flag: None - Use random weights
#                      String - Use pretrained weights given by String
use_pretrained = None

# Initialize the model for this run.
model_pyt, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained)

# lr start and end points for training.
lr_start = 0.01
lr_end = 0.001

# How many epochs to restart.
iter_restart = 10

# Multiplication factor after restart.
mul_restart = 1

# Print the model we just instantiated
#print(model_ft)

In [40]:
data_loaders = load_data(images_dir,
                         batch_size = batch_size, 
                         input_size = (input_size, input_size), 
                         norm_arr = norm_arr,
                         num_workers = num_workers)

plot_data_loader(data_loader['train'], (2,2))

plot_data_loader(data_loader['test'], (2,2))

plot_data_loader(data_loader['val'], (2,2))

In [41]:
# Check if GPU is available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Send model to GPU
model_pyt = model_pyt.to(device)

# Find parameters to be updated in this run.
# parameters with requires_grad = True.
params_to_update = model_pyt.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

cuda:0
Params to learn:
	 features.0.0.weight
	 features.0.1.weight
	 features.0.1.bias
	 features.1.0.block.0.0.weight
	 features.1.0.block.0.1.weight
	 features.1.0.block.0.1.bias
	 features.1.0.block.1.fc1.weight
	 features.1.0.block.1.fc1.bias
	 features.1.0.block.1.fc2.weight
	 features.1.0.block.1.fc2.bias
	 features.1.0.block.2.0.weight
	 features.1.0.block.2.1.weight
	 features.1.0.block.2.1.bias
	 features.1.1.block.0.0.weight
	 features.1.1.block.0.1.weight
	 features.1.1.block.0.1.bias
	 features.1.1.block.1.fc1.weight
	 features.1.1.block.1.fc1.bias
	 features.1.1.block.1.fc2.weight
	 features.1.1.block.1.fc2.bias
	 features.1.1.block.2.0.weight
	 features.1.1.block.2.1.weight
	 features.1.1.block.2.1.bias
	 features.2.0.block.0.0.weight
	 features.2.0.block.0.1.weight
	 features.2.0.block.0.1.bias
	 features.2.0.block.1.0.weight
	 features.2.0.block.1.1.weight
	 features.2.0.block.1.1.bias
	 features.2.0.block.2.fc1.weight
	 features.2.0.block.2.fc1.bias
	 features.2.0.blo

In [42]:
# conda install -c iopath iopath 
# conda install -c conda-forge yacs  
# conda install -c fvcore -c iopath -c conda-forge fvcore

In [43]:
from fvcore.nn import FlopCountAnalysis

In [20]:
# mobile_net_v3_large
flops_count = 0
for inputs, labels in data_loaders['train']:
    inputs = inputs.to(device)
    flops = FlopCountAnalysis(model_pyt, inputs)
    flops_count = flops.total()
    print(f"{flops_count:,}")
    break

Unsupported operator aten::add_ encountered 56 time(s)
Unsupported operator aten::hardswish_ encountered 21 time(s)
Unsupported operator aten::hardsigmoid encountered 8 time(s)
Unsupported operator aten::mul encountered 8 time(s)
Unsupported operator aten::dropout_ encountered 1 time(s)


4,370,335,872


In [45]:
# efficient b1
flops_count = 0
for inputs, labels in data_loaders['train']:
    inputs = inputs.to(device)
    flops = FlopCountAnalysis(model_pyt, inputs)
    flops_count = flops.total()
    print(f"{flops_count:,}")
    break

Unsupported operator aten::add_ encountered 85 time(s)
Unsupported operator aten::silu_ encountered 69 time(s)
Unsupported operator aten::sigmoid encountered 23 time(s)
Unsupported operator aten::mul encountered 39 time(s)
Unsupported operator aten::bernoulli_ encountered 16 time(s)
Unsupported operator aten::div_ encountered 16 time(s)
Unsupported operator aten::dropout_ encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
features.1.0.stochastic_depth, features.2.0.stochastic_depth, features.3.0.stochastic_depth, features.4.0.stochastic_depth, features.5.0.stochastic_depth, features.6.0.stochastic_depth, features.7.0.stochastic_depth


12,923,375,616


In [32]:
# resnet34
flops_count = 0
for inputs, labels in data_loaders['train']:
    inputs = inputs.to(device)
    flops = FlopCountAnalysis(model_pyt, inputs)
    flops_count = flops.total()
    print(f"{flops_count:,}")
    break

Unsupported operator aten::add_ encountered 52 time(s)
Unsupported operator aten::max_pool2d encountered 1 time(s)


76,945,580,032
